In [12]:
import math
import nbimporter
from CoolProp.CoolProp import PropsSI

In [13]:
def F2K(F):
    return (F - 32) * 5/9 + 273.15

### Problem 1: Duct heat loss
A duct that is 2 ft by 5 ft in cross-section and 80 ft long carries conditioned air through the space above the hung ceiling in an office building. The air enters the duct from the air handler at 55 F and 2 in. water pressure with a flow rate of 8,000 cfm. The temperature in the space is 85 F. Determine\
a. The level of insulation needed on the sides of the duct to have a temperature rise of 1 F\
b. The drop in sensible cooling capacity due to the heat transfer to the air flow.\
c. The leakage flow rate from the duct.\
d. The total drop in capacity due to leakage and heat loss in energy and relative to the total capacity.\
Assume you are using insulation with a thermal
conductivity of kins = 0.025 Btu/hr-ft-F. Also assume a duct leakage class of 6.

In [14]:
T_i = 55 #[F]
T_o = 56
T_a = 85
V_dot = 8000
L = 80
Area = 2 * (2 + 5) * L

In [15]:
P_atm = 14.7 #[psi]
rho_air = PropsSI('D', 'T', F2K(T_i), 'P', 100e5, 'Air')
c_p = PropsSI('C', 'T', F2K(T_i), 'P', 100e5, 'Air')
print(rho_air)

124.02814921997911
